In [1]:
import psycopg2
from configparser import ConfigParser
from pandas import DataFrame
from collections import Counter

In [2]:
cfg = ConfigParser()
cfg.read("db.cfg")
knst = psycopg2.connect(host=cfg['db']['host'], port=cfg['db']['port'],
                        database=cfg['db']['db'], user=cfg['db']['user'],
                        password=cfg['db']['pwd'])
knst.set_client_encoding('UTF-8')
cur = knst.cursor()

/Users/joachimbovin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  


In [3]:
sql = """

SELECT DISTINCT org.id, org.name,
rel_org_date_creation.day, rel_org_date_creation.month, rel_org_date_creation.year , 
rel_org_date_cancel.day, rel_org_date_cancel.month, rel_org_date_cancel.year ,
rel_org_date_start_act.day, rel_org_date_start_act.month, rel_org_date_start_act.year, 
rel_org_date_end_act.day, rel_org_date_end_act.month, rel_org_date_end_act.year,
rel_org_countries.name_nl, org.archive_url, 
rel_org_location.zip_code, org.url, rel_org_location.city_nl


FROM production.organisations AS org

LEFT JOIN production.countries AS rel_org_countries
ON org.country_id = rel_org_countries.id

LEFT JOIN production.date_isaars AS rel_org_date_creation
ON org.creation_date_id = rel_org_date_creation.id

LEFT JOIN production.date_isaars AS rel_org_date_cancel
ON org.cancellation_date_id = rel_org_date_cancel.id

LEFT JOIN production.date_isaars AS rel_org_date_start_act
ON org.start_activities_date_id = rel_org_date_start_act.id

LEFT JOIN production.date_isaars AS rel_org_date_end_act
ON org.end_activities_date_id = rel_org_date_end_act.id

LEFT JOIN production.locations AS rel_org_location
ON org.location_id = rel_org_location.id

LEFT JOIN production.organisation_types_organisations AS org_types
ON org.id = org_types.organisation_id

LEFT JOIN production.relationships as rel
ON rel.organisation_id = org.id


WHERE org_types.organisation_id IS NULL AND rel.production_id IS NOT NULL AND org.country_id = 12928 

"""

In [4]:
from datetime import date
from pandas import isnull

cur.execute(sql)
organisations = cur.fetchall()
df = DataFrame(organisations, columns = ["ID","Organisation_name", "creation day", "creation month", "creation year", "end_day", "end_month", "end_year", "start_act_day", 
                                        "start_act_month", "start_act_year", "end_act_day","end_act_month","end_act_year", "country", "archief_website", "zip_code", "website", "stad"])

df['Creation_date'] = df[df.columns[2:5]].apply(lambda x: date(int(x[2]) if not isnull(x[2]) else 1,
                                                               int(x[1]) if not isnull(x[1]) else 1,
                                                               int(x[0]) if not isnull(x[0]) else 1), axis=1)
df['End_date'] = df[df.columns[5:8]].apply(lambda x: date(int(x[2]) if not isnull(x[2]) else 1,
                                                               int(x[1]) if not isnull(x[1]) else 1,
                                                               int(x[0]) if not isnull(x[0]) else 1), axis=1)
df['Start_activities_date'] = df[df.columns[8:11]].apply(lambda x: date(int(x[2]) if not isnull(x[2]) else 1,
                                                               int(x[1]) if not isnull(x[1]) else 1,
                                                               int(x[0]) if not isnull(x[0]) else 1), axis=1)
df['End_activities_date'] = df[df.columns[11:14]].apply(lambda x: date(int(x[2]) if not isnull(x[2]) else 1,
                                                               int(x[1]) if not isnull(x[1]) else 1,
                                                               int(x[0]) if not isnull(x[0]) else 1), axis=1)

df = df.drop(['creation day', 'creation month', 'creation year', "end_day", "end_month", "end_year", "start_act_day", 
                                        "start_act_month", "start_act_year", "end_act_day","end_act_month","end_act_year"], axis=1)



df.head(n=30)

#print len(df["Organisation_name"].unique())
#print len(df)

,ID,Organisation_name,country,archief_website,zip_code,website,stad,Creation_date,End_date,Start_activities_date,End_activities_date
0,374847,Vocaal ensemble Incensum,België,None,None,http://www.ensemble-incensum.be,Hofstade (Bt.),2013-01-01,0001-01-01,0001-01-01,0001-01-01
1,373256,Theater Tartaar,België,None,None,http://www.theatertartaar.be/home,Lennik - Eizeringen,0001-01-01,0001-01-01,0001-01-01,0001-01-01
2,383500,Huize Eigen Haard vzw (Apojo),België,,3200,https://www.apojo.be,Aarschot,0001-01-01,0001-01-01,0001-01-01,0001-01-01
3,382069,Texture Museum,België,,8500,,Kortrijk,0001-01-01,0001-01-01,0001-01-01,0001-01-01
4,371555,Museum Plantin-Moretus,België,None,None,http://museum.antwerpen.be/plantin_moretus,Antwerpen,0001-01-01,0001-01-01,0001-01-01,0001-01-01
5,364172,Théâtre National de la Communauté française,België,None,1210,http://www.theatrenational.be,Brussel,0001-01-01,0001-01-01,0001-01-01,0001-01-01
6,373899,Steunpunt Tewerkstelling,België,None,2800,,Mechelen,0001-01-01,0001-01-01,0001-01-01,0001-01-01
7,362296,WERKGROEP S.J.T.,België,None,1652,None,Alsemberg,0001-01-01,0001-01-01,0001-01-01,0001-01-01
8,363208,Het Laatste kwartier,België,None,None,None,None,1997-08-01,0001-01-01,0001-01-01,0001-01-01
9,375275,MAfestival,België,,None,http://www.mafestival.be/,None,0001-01-01,0001-01-01,0001-01-01,0001-01-01


In [5]:
import pandas as pd

doubles = df.duplicated("Organisation_name")

df_doubles = doubles.to_frame()

index_doubles = []

count = 0

for i in range(len(df_doubles)):
    if doubles[i] == True:
        index_doubles.append(i)

print index_doubles

for index in index_doubles:
    print df["Organisation_name"][index]

[1486, 1654, 1867, 2073, 2454]
Kasteel Mariagaarde
PASS
CC Maasmechelen
BIN, Belgisch Instituut voor Normalisatie
BALLET THEATRE L'ENSEMBLE


In [6]:
#juni heeft maar 30 dagen! 

print df.ix[542]

ID                                               382243
Organisation_name        Ministère de la Région Wallone
country                                          België
archief_website                                        
zip_code                                           9300
website                                                
stad                                              Aalst
Creation_date                                0001-01-01
End_date                                     0001-01-01
Start_activities_date                        0001-01-01
End_activities_date                          0001-01-01
Name: 542, dtype: object


/Users/joachimbovin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
sql_2 = """

SELECT DISTINCT org_from.id, org_from.name, org_rel_types.from_name_nl, org_to.id, org_to.name

FROM production.organisations AS org_from

LEFT JOIN production.organisation_relations org_rel
ON org_from.id = org_rel.from_id

LEFT JOIN production.organisations AS org_to
ON org_rel.to_id = org_to.id

LEFT JOIN production.organisation_relation_types AS org_rel_types
ON org_rel_types.id = org_rel.organisation_relation_type_id

LEFT JOIN production.organisation_types_organisations AS org_types
ON org_from.id = org_types.organisation_id

LEFT JOIN production.relationships as rel
ON rel.organisation_id = org_from.id

WHERE org_types.organisation_id IS NULL AND rel.production_id IS NOT NULL AND org_from.country_id = 12928 

"""




In [8]:
cur.execute(sql_2)
organisations = cur.fetchall()
df_2 = DataFrame(organisations, columns=["from_id", "from_name", "relation_name", "to_id", "to_name"])

print len(df_2["from_id"].unique())


#df_2.head(n=30)

df_2.head(n=30)


2542


,from_id,from_name,relation_name,to_id,to_name
0,360575,het ro theater,None,NaN,None
1,360576,Het Affront / Amana Dance Theatre,None,NaN,None
2,360577,Agora,None,NaN,None
3,360578,Akt-vertikaal,Voortzetting van,363261.0,Theater Vertikaal
4,360579,Alan produkties vzw,None,NaN,None
5,360581,Théâtre de l'Ancre,None,NaN,None
6,360582,Theater Antigone,None,NaN,None
7,360584,Arca,Voortzetting van,371247.0,Toneelstudio '50
8,360586,Atelier Ste. Anne,None,NaN,None
9,360587,Atelier théâtral de Louvain-la-Neuve,None,NaN,None


In [127]:
relations_between_orgs = []
for row in df_2.iterrows():
    from_name = row[1]["from_id"]
    relation = row[1]["relation_name"]
    to_name = row[1]["to_name"]
    relations_between_orgs.append({"index": from_name, relation: to_name})
df_2_relations = DataFrame(relations_between_orgs)
df_2_relations_nafilled_and_index_resetted = df_2_relations.fillna(0).reset_index().drop(["level_0", None], axis=1)
df_2_relations_nafilled_and_index_resetted.head()

,Afsplitsing van,Festival van,Fusie van,Onderdeel van,Soirée Composée van,Voortzetting van,Werkt samen met,index
0,0,0,0,0,0,0,0,360575
1,0,0,0,0,0,0,0,360576
2,0,0,0,0,0,0,0,360577
3,0,0,0,0,0,Theater Vertikaal,0,360578
4,0,0,0,0,0,0,0,360579


In [10]:
print len(df_2_relations_nafilled_and_index_resetted["index"].unique())

2542


In [11]:
sql3 = """

SELECT org_rel.from_name_nl, org_rel.to_name_nl, org_rel.id

FROM production.organisation_relation_types AS org_rel

"""

#

In [12]:
cur.execute(sql3)
organisations = cur.fetchall()
df_3 = DataFrame(organisations)

print df_3

                     0                          1    2
0        Onderdeel van        Heeft als onderdeel  124
1      Afsplitsing van               Gesplitst in  125
2     Voortzetting van             Voortgezet als  127
3         Festival van         Heeft als festival  128
4      Werkt samen met            Werkt samen met  129
5  Soirée Composée van  heeft als Soirée Composée  130
6            Fusie van            Gefusioneerd in  126


In [47]:
sql4 = """

SELECT org_rel.id, org_rel.from_id

FROM production.organisation_relations AS org_rel

"""

In [48]:
cur.execute(sql4)
organisations = cur.fetchall()
df_4 = DataFrame(organisations)


In [129]:
dfinal = df.merge(df_2_relations_nafilled_and_index_resetted, how='inner', left_on="ID", right_on='index')


dfinal= dfinal.drop("index", axis=1)
dfinal= dfinal.drop("Fusie van", axis=1)
dfinal = dfinal.drop("Soirée Composée van", axis=1)
dfinal = dfinal.drop("archief_website", axis=1)


dfinal[dfinal["ID"] == 360654]

,ID,Organisation_name,country,zip_code,website,stad,Creation_date,End_date,Start_activities_date,End_activities_date,Afsplitsing van,Festival van,Onderdeel van,Voortzetting van,Werkt samen met
366,360654,Theater Froe Froe,België,None,http://www.froefroe.be,Antwerpen,1997-01-01,0001-01-01,0001-01-01,0001-01-01,0,0,0,FIGURENTHEATER BLACK OUT,0
367,360654,Theater Froe Froe,België,None,http://www.froefroe.be,Antwerpen,1997-01-01,0001-01-01,0001-01-01,0001-01-01,0,0,0,Blauw Vier,0
368,360654,Theater Froe Froe,België,None,http://www.froefroe.be,Antwerpen,1997-01-01,0001-01-01,0001-01-01,0001-01-01,0,0,0,0,HETPALEIS


In [121]:
for index, serie in dfinal.iterrows():
    print index
    for kolomnaam in serie.index:
        value = serie[kolomnaam]
        if isinstance(value, list) and len(value) == 1:
            print "changing", index, kolomnaam, value
            dfinal.at[index, kolomnaam] = value[0]
        if isinstance(value, list) and len(value) > 1:
            #dfinal.drop(index, axis=0, inplace=True)
            new_df = DataFrame([serie]*len(value)).reset_index(drop=True)
            for new_df_index, new_df_row in new_df.iterrows():
                new_df.at[new_df_index, kolomnaam] = value[new_df_index]
            print new_df
            dfinal = dfinal.append(new_df, ignore_index=True)
            


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
changing 60 Onderdeel van ['Fakkelteater']
61
62
63
64
changing 64 Voortzetting van ['CC Merelbeke']
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
changing 93 Voortzetting van ['Mechels Miniatuur Teater']
94
changing 94 Werkt samen met ['Verenigde werkhuizen Thassos v.z.w.']
95
96
changing 96 Festival van ['Villanella']
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
changing 141 Voortzetting van ['Nederlands Toneel Gent']
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
changing 166 Voortzetting van ["Toneelstudio '50"]
167
168
169
170
171
172
173
174
175
176
changing 176 Voortzetting van ['Theater

changing 988 Werkt samen met ['Kolenspoorfestival']
989
changing 989 Voortzetting van ["CC 't Ghelandt"]
990
991
992
993
994
995
996
997
changing 997 Afsplitsing van ['De Onderneming']
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
changing 1032 Onderdeel van ['Muzi a Zeny']
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
changing 1088 Onderdeel van [None]
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
changing 1116 Onderdeel van ['Muziektheater Transparant']
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133


2227
2228
changing 2228 Werkt samen met ['Foton']
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
changing 2245 Voortzetting van ['Het Muziek LOD']
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
       ID Organisation_name country zip_code website  stad Creation_date  \
0  369641        Dubbelspel  België     None    None  None    0001-01-01   
1  369641        Dubbelspel  België     None    None  None    0001-01-01   

     End_date Start_activities_date End_activities_date  Afsplitsing van  \
0  0001-01-01            0001-01-01          0001-01-01                0   
1  0001-01-01            0001-01-01          0001-01-01                0   

   Festival van  Onderdeel van  Voortzetting van          Werkt samen met  
0             0              0                 0  STUK Kunstencentrum vzw  
1             0              0                 0                     30CC  
2256
2257
2258
2259
2260
2261
2262
2263
changing 2263 Voortzetting van ['Poppen

In [130]:
dfinal.to_csv("kunstenpunt_organisaties_metadata_test_2.csv")